In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, smooth_landmarks=True, 
                     min_detection_confidence=0.5, min_tracking_confidence=0.5)

cap = cv2.VideoCapture('dance_opencb.mp4')

POSE_CONNECTIONS = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), 
                    (11, 23), (12, 24), (23, 24), (23, 25), (24, 26), 
                    (25, 27), (26, 28), (27, 31), (28, 32)]

colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255)]
color_index = 0
last_color_change = time.time()

while cap.isOpened():
    ret, frame = cap.read() # true or false and frame
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if time.time() - last_color_change > 3:
        color_index = (color_index + 1) % len(colors)
        last_color_change = time.time()

    glow_color = colors[color_index]

    if results.pose_landmarks:
        height, width, _ = frame.shape
        center_x = width // 2
        clone_offset = width // 4

        landmarks_2d = [(int(lm.x * width), int(lm.y * height)) for lm in results.pose_landmarks.landmark]
        mirror_2d = [(center_x - (x - center_x) + clone_offset, y) for x, y in landmarks_2d]

        glow_layer = np.zeros_like(frame, dtype=np.uint8)

        for i, j in POSE_CONNECTIONS:
            cv2.line(glow_layer, mirror_2d[i], mirror_2d[j], glow_color, 10)
            cv2.line(glow_layer, mirror_2d[i], mirror_2d[j], (255, 255, 255), 3)

        for x, y in mirror_2d:
            cv2.circle(glow_layer, (x, y), 10, glow_color, -1)
            cv2.circle(glow_layer, (x, y), 5, (255, 255, 255), -1)

        frame = cv2.addWeighted(frame, 0.8, glow_layer, 0.6, 0)

    cv2.imshow("Dance", frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [17]:
clone_offset

160

In [18]:
width

640

In [19]:
center_x

320

In [26]:
for lm in results.pose_landmarks.landmark:
    print(lm.x, lm.y, lm.z)


0.5058504939079285 0.172905832529068 -0.275096595287323
0.511618971824646 0.162033811211586 -0.25291553139686584
0.5146483778953552 0.16345162689685822 -0.25294721126556396
0.5179247856140137 0.16509409248828888 -0.2530401349067688
0.5026994347572327 0.16045187413692474 -0.24931983649730682
0.499540239572525 0.16074030101299286 -0.24934282898902893
0.4961516857147217 0.16133752465248108 -0.24934883415699005
0.5217745900154114 0.1779783070087433 -0.1406739056110382
0.49262815713882446 0.17232157289981842 -0.12633207440376282
0.5110248327255249 0.19228261709213257 -0.23443815112113953
0.49978891015052795 0.1891445517539978 -0.23054960370063782
0.5337488651275635 0.26656249165534973 -0.13383492827415466
0.4662957787513733 0.2607050836086273 -0.07784565538167953
0.5554636716842651 0.1726515293121338 -0.23170483112335205
0.44772258400917053 0.14501163363456726 -0.14595220983028412
0.5602480173110962 0.09183669090270996 -0.32469338178634644
0.4426625370979309 0.07621248066425323 -0.261037528